<a href="https://colab.research.google.com/github/ClaudioLucioLopes/PrescriptiveAnalysis/blob/master/2_0_Projeto_de_Investimento_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso da Projeto de investimentos HostsM

# Instalação e módulos necessários

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp
import numpy as np


# Organizando os dados

In [0]:
#Número de projetos
num_projetos = 7

#Valor total dos projetos por ano
cronograma_desembolso = np.array([[12, 34, 12],
                                  [54, 94, 67],
                                  [65, 28, 49],
                                  [38, 0, 8],
                                  [52, 21, 42],
                                  [98, 73, 25],
                                  [15, 48, 53]])

#VPL dos projetos 
VPL_Projeto = np.array([20, 15, 34, 17, 56, 76, 29])

#Valor máximo de desembolso por ano
Previsto_desembolso = np.array([200 ,250, 150])


# Criando o modelo

In [0]:
# Criando o modelo de investimento
modelo_projetos_inv = pywraplp.Solver('Investimentos HotsM',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

#Criando as variáveis de decisão
projetos = {}
for i in range(num_projetos):
    projetos[i] = modelo_projetos_inv.NumVar(0, 1, 'projetos[%i]' % (i))
print(projetos)

#Criando a nossa função objetivo
total_VPL =  modelo_projetos_inv.Sum([VPL_Projeto[i] * projetos[i]   for i in range(num_projetos)])    

#Atribuindo a maximização
objetivo = modelo_projetos_inv.Maximize(total_VPL)



#Restrições para os cronogramas de desembolso e projetos

#Restrição do montante para o Ano 1
modelo_projetos_inv.Add(modelo_projetos_inv.Sum(cronograma_desembolso[i,0] *projetos[i]  for i in range(num_projetos)) <= Previsto_desembolso[0])

#Restrição do montante para o Ano 2
modelo_projetos_inv.Add(modelo_projetos_inv.Sum(cronograma_desembolso[i,1] *projetos[i]  for i in range(num_projetos)) <= Previsto_desembolso[1])

#Restrição do montante para o Ano 3
modelo_projetos_inv.Add(modelo_projetos_inv.Sum(cronograma_desembolso[i,2] *projetos[i]  for i in range(num_projetos)) <= Previsto_desembolso[2])

#Já fizemos, mas só para deixar explicito aqui a restrição do percentual de partipação
for i in range(num_projetos):
    modelo_projetos_inv.Add(projetos[i] >= 0 )
    modelo_projetos_inv.Add(projetos[i] <= 1 )



{0: projetos[0], 1: projetos[1], 2: projetos[2], 3: projetos[3], 4: projetos[4], 5: projetos[5], 6: projetos[6]}


# Executando o modelo

In [0]:
#Executando o modelo
modelo_projetos_inv.EnableOutput = True
modelo_projetos_inv.Solve()

print('Tempo: ', modelo_projetos_inv.WallTime(), 'ms')
print('Iterações: ', modelo_projetos_inv.iterations())

Tempo:  864782 ms
Iterações:  0


# Avaliando os resultados

In [0]:
print('Valor máximo do VPL a ser obtido: ', round(modelo_projetos_inv.Objective().Value(),2))

for i in range(num_projetos):
    print('Projeto %i:' % (i+1), round(projetos[i].SolutionValue() *100,2),'%')


Valor máximo do VPL a ser obtido:  193.03
Projeto 1: 100.0 %
Projeto 2: 0.0 %
Projeto 3: 35.38 %
Projeto 4: 0.0 %
Projeto 5: 100.0 %
Projeto 6: 100.0 %
Projeto 7: 100.0 %
